[Markdown Guide](https://docs.gitlab.com/ee/user/markdown.html)

# Praxis Modul I
## SoSe 2022 - HS Mainz

### Sozial Network Analyse

Untersuchen von sozialen Netzwerken mithilfe von PM4PY.

Durchgeführt von

| Student | Matrikel-Nummer |
|:-- |:--|
|Moritz Wetzel | 123456|
|Till Waller | 123456|
Fabian Harmsen| 123456|



## Erklärung der Grafischen Ergebnisse der `Similar Activities` und `Handover of Work`-Analysen
*Wir erzeugen beim bzw nach dem Clustering keine Grafiken, da dies in den Getestetn Datensätzen keinen Mehrwert erzeugt hätte*

Bei den obengenannten Methoden werden Grafische Ausgaben erzeugt.
Diese Ausgaben haben zwei Basis Komponenten, Nodes (Knoten) und Edges (Verbindungen).

In unseren Analysen ist ein Node immer ein Nutzer (bzw. nach dem Clustering eine Benutzergruppe).
Ein Edge ist hingegen die Verbindung zwischen den einzelnen Nodes. Die Stärke der einzelnen Nodes wird durch die Analysen bestimmt und entsprechend angewendet.

Auf dieser Grundlage wird dann eine Physiksimulation erstellt.
Dabei wirken die Edges wie Gummibänder, die die Nodes zusammenhalten. Während die Nodes versuchen möglichst viel Abstand zu anderen Nodes zu haben.
Diese Konstellation führt dazu, dass Nodes die aufgrund der Analyse mit starken Edges verbunden sind zusammenrücken und wenige stark verbunde Nodes auseinander treiben.
Sobald sich die Physiksimulation beruhigt hat, kann man so Gruppen von Nodes erkennen und auch die Verbindungsnodes zu anderen Gruppen,

# Configuration
Die unten stehende Table erklärt die Funktion der Variablen im nächsten Code-Block.

| variable | type | description| default|
|:--|:--|:--|:--|
|log_path | (path)| path to the XES File|`data/test.xes`|
|use_networkx | (bool)| output via NetworkX or Pyvis| `False`|
|filter_timeframe | (bool)| filter the timeframe of the traces| `False` |
|filter_timeframe_intersecting | (bool)| use intersecting or contained| `False` |
|filter_timeframe_start | (dateTime)| start of the Timeframe| `1970-01-01 00:00:00` |
|filter_timeframe_end | (dateTime)| end of the Timeframe| `2038-12-31 00:00:00` |
|filter_performance| (bool)| filter on Traceperformance| `False` |
|filter_performance_min| (int)| minimum seconds of trace actions|`0` |
|filter_performance_max |(int)| maximum seconds of trace actions|`360000` |

Die Option `Pyvis` legt die Ergebnisse im Ordner `./result` ab, dieser muss beschreibbar sein.

In [72]:
log_path = 'data/BPI_Challenge_2017.xes'

use_networkx = False

filter_timeframe = False
filter_timeframe_intersecting  = False
filter_timeframe_start = "2011-03-09 00:00:00"
filter_timeframe_end = "2011-03-09 00:00:00"

filter_performance = False
filter_performance_min = 0
filter_performance_max = 864000

#### Import
Im folgenden Codeblock werden alle Bibliotheken geladen die im laufenden Script benötigt werden.
Wenn diese Bibliotheken nicht geladen werden können, wird der Script nicht funktionieren.

**Unvollständige Liste der Pythonpackete (pip)**
 - pm4py
 - shutil
 - numpy
 - sklearn
 - scipy


In [73]:
import shutil
from pm4py.objects.log.importer.xes import importer as xes_importer

from pm4py.algo.organizational_mining.sna import algorithm as sna
from pm4py.visualization.sna import visualizer as sna_visualizer
from pm4py.algo.organizational_mining.roles import algorithm as roles_discovery
from pm4py.algo.organizational_mining.sna import util
from pm4py.algo.filtering.log.timestamp import timestamp_filter
from pm4py.algo.filtering.log.cases import case_filter
import pm4py
import os
from pm4py.algo.organizational_mining.local_diagnostics import algorithm as local_diagnostics

#### Allgemeiner Importskript
Läd die in der Konfig angegebenen Datensatz und wendet die eingestellten Filter an.

In [74]:
log = xes_importer.apply(log_path)

if filter_timeframe:
    if not filter_timeframe_intersecting:
        log = timestamp_filter.filter_traces_contained(   log, filter_timeframe_start, filter_timeframe_end)
    else:
        log = timestamp_filter.filter_traces_intersecting(log, filter_timeframe_start, filter_timeframe_end)

if filter_performance:
    log = case_filter.filter_case_performance(log, filter_performance_min, filter_performance_max)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

**Similar Activities**
Berechtet wie ähnlich die aktivitäten der Benutzer sind

Ergebnis: result/similar_activities.html

In [75]:
sa_metric  = sna.apply(log, variant=sna.Variants.JOINTACTIVITIES_LOG)

if use_networkx:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_ja_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_ja_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_ja_py, "result/similar_activities.html")

Discover Roles

Erkennt die Rollen eines Benutzers anhand der durchgeführten Aktivitäten.
Nutzer, die dieselben Aktivitäten werden zusammen gruppiert.

**Ausgabeformat**
1. Rollenname auf basis der Aktivitäten
2. Benutzer mit der Angabe wie aktiv der Nutzer in der Rolle war
3. `Blank Row`

In [76]:
roles = roles_discovery.apply(log)

for x in roles:
    print(x[0])
    print(x[1])
    print("")

['A_Accepted', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'W_Complete application']
{'User_52': 4916, 'User_38': 4084, 'User_19': 5039, 'User_17': 4555, 'User_3': 11443, 'User_31': 2061, 'User_43': 3157, 'User_16': 5681, 'User_21': 4145, 'User_15': 5692, 'User_108': 586, 'User_89': 196, 'User_28': 8097, 'User_7': 3600, 'User_41': 6064, 'User_22': 1160, 'User_110': 75, 'User_10': 10090, 'User_11': 3059, 'User_13': 2570, 'User_20': 1018, 'User_37': 6965, 'User_71': 6505, 'User_74': 1532, 'User_8': 522, 'User_5': 8396, 'User_14': 5479, 'User_4': 6170, 'User_75': 251, 'User_2': 5111, 'User_24': 5066, 'User_92': 422, 'User_85': 1911, 'User_6': 288, 'User_44': 3865, 'User_27': 3419, 'User_49': 12623, 'User_36': 4412, 'User_47': 1550, 'User_35': 5790, 'User_30': 108, 'User_87': 213, 'User_25': 1941, 'User_18': 7642, 'User_12': 4627, 'User_9': 600, 'User_34': 1837, 'User_33': 1691, 'User_26': 2990, 'User_29': 222, 'User_95': 103, 'User_50': 372, 'User_70': 2653, '

Handover of Work

In [77]:
hw_values = sna.apply(log, variant=sna.Variants.HANDOVER_LOG)

if use_networkx:
    gviz_hw_py = sna_visualizer.apply(hw_values, variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_hw_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_hw_py = sna_visualizer.apply(sa_metric , variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_hw_py, "result/handover_of_work.html")

CLuster Similar Activities into groups
Clustering the results of the similar activities metric, individuals that work on the same tasks would be inserted in the same group.

**Ausgabeformat**
1. Cluster
2. Benutzerliste
3. `Blank Row`

In [78]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)

for group in clustering_sa:
    print(group)
    print(clustering_sa[group])
    print("")


3
['User_1', 'User_138', 'User_143', 'User_144', 'User_147']

6
['User_10', 'User_13', 'User_141', 'User_15', 'User_16', 'User_17', 'User_18', 'User_19', 'User_21', 'User_22', 'User_24', 'User_35', 'User_36', 'User_37', 'User_38', 'User_4', 'User_40', 'User_42', 'User_43', 'User_44', 'User_45', 'User_46', 'User_47', 'User_49', 'User_51', 'User_52', 'User_60', 'User_61', 'User_62', 'User_70', 'User_74', 'User_84', 'User_85', 'User_88', 'User_92']

0
['User_100', 'User_101', 'User_102', 'User_107', 'User_109', 'User_115', 'User_124', 'User_128', 'User_129', 'User_136', 'User_148', 'User_29', 'User_75', 'User_90', 'User_99']

4
['User_103', 'User_104', 'User_105', 'User_108', 'User_110', 'User_12', 'User_132', 'User_135', 'User_20', 'User_25', 'User_32', 'User_50', 'User_55', 'User_56', 'User_57', 'User_59', 'User_6', 'User_65', 'User_69', 'User_7', 'User_71', 'User_72', 'User_73', 'User_76', 'User_77', 'User_78', 'User_79', 'User_8', 'User_80', 'User_81', 'User_82', 'User_89', 'User_9', 

Clustering the results of the working together metric, individuals that work often together would be inserted in the same group.

**Ausgabeformat**
1. Cluster
2. Benutzerliste
3. `Blank Row`

In [79]:
clustering_hw = util.cluster_affinity_propagation(hw_values)
for group in clustering_hw:
    print(group)
    print(clustering_hw[group])
    print("")

0
['User_1']

1
['User_10']

2
['User_100']

15
['User_101', 'User_102', 'User_103', 'User_104', 'User_105', 'User_106', 'User_107', 'User_108', 'User_11', 'User_110', 'User_111', 'User_114', 'User_115', 'User_119', 'User_12', 'User_122', 'User_124', 'User_125', 'User_127', 'User_128', 'User_129', 'User_13', 'User_130', 'User_131', 'User_132', 'User_134', 'User_135', 'User_136', 'User_137', 'User_138', 'User_139', 'User_140', 'User_141', 'User_142', 'User_143', 'User_144', 'User_145', 'User_146', 'User_147', 'User_148', 'User_149', 'User_17', 'User_19', 'User_20', 'User_21', 'User_22', 'User_23', 'User_24', 'User_25', 'User_26', 'User_31', 'User_32', 'User_33', 'User_34', 'User_35', 'User_36', 'User_38', 'User_39', 'User_43', 'User_44', 'User_45', 'User_47', 'User_48', 'User_50', 'User_51', 'User_52', 'User_53', 'User_54', 'User_55', 'User_56', 'User_57', 'User_58', 'User_59', 'User_6', 'User_60', 'User_62', 'User_63', 'User_64', 'User_65', 'User_66', 'User_67', 'User_69', 'User_7', 'U

In [80]:
# this applies the organizational mining from an attribute that is in each event, describing the group that is performing the task.
ld = local_diagnostics.apply_from_group_attribute(log)
# GROUP RELATIVE FOCUS (on a given type of work) specifies how much a resource group performed this type of work
# compared to the overall workload of the group. It can be used to measure how the workload of a resource group
# is distributed over different types of work, i.e., work diversification of the group.
print("\ngroup_relative_focus")
print(ld["group_relative_focus"])
# GROUP RELATIVE STAKE (in a given type of work) specifies how much this type of work was performed by a certain
# resource group among all groups. It can be used to measure how the workload devoted to a certain type of work is
# distributed over resource groups in an organizational model, i.e., work participation by different groups.
print("\ngroup_relative_stake")
print(ld["group_relative_stake"])
# GROUP COVERAGE with respect to a given type of work specifies the proportion of members of a resource group that
# performed this type of work.
print("\ngroup_coverage")
print(ld["group_coverage"])
# GROUP MEMBER CONTRIBUTION of a member of a resource group with respect to the given type of work specifies how
# much of this type of work by the group was performed by the member. It can be used to measure how the workload
# of the entire group devoted to a certain type of work is distributed over the group members.
print("\ngroup_member_contribution")
print(ld["group_member_contribution"])


group_relative_focus
{}

group_relative_stake
{}

group_coverage
{}

group_member_contribution
{}
